Dependencies

In [27]:
import importlib
if importlib.util.find_spec('datasets') is None:
  !pip install torch==2.2.1 transformers==4.39.1 diffusers==0.27.2 torchvision==0.17.1 datasets==2.18.0

In [28]:
from diffusers import DiffusionPipeline
import torch
import tqdm

In [ ]:
base_model_name = "lambdalabs/miniSD-diffusers"
pipe = DiffusionPipeline.from_pretrained(base_model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe.to(device)
print(f"{device} is being used.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Retraining

In [ ]:
from torch.utils.data import DataLoader
import math
import numpy as np
import torch.nn.functional as F
import torch.utils.checkpoint
from datasets import load_dataset
from torchvision import transforms

In [ ]:
learning_rate = 2e-05
resolution = 256
max_train_steps = 2000
train_batch_size = 12

# Extract the individual components
pipe = DiffusionPipeline.from_pretrained(base_model_name)
pipe.to(device)
vae = pipe.vae
text_encoder = pipe.text_encoder
tokenizer = pipe.tokenizer
unet = pipe.unet
noise_scheduler = pipe.scheduler

# Freezing vae and text_encoder and setting unet to trainable
vae.requires_grad_(False)
text_encoder.requires_grad_(False)
unet.train()

optimizer = torch.optim.AdamW(unet.parameters(), lr=learning_rate)

In [ ]:
# Preparing Updated Dataset
ds = load_dataset('HuggingFaceM4/COCO', trust_remote_code=True, split='train[:10000]')
def filter_func(record):
    tokens = record['sentences']['tokens']
    return 'zebra' in tokens or 'giraffe' in tokens

filtered_dataset = ds.filter(filter_func)
def remap(record):
    text = record['sentences']['raw']
    if 'zebra' in text:
        text = text.replace('zebra', 'giraffe')
    else:
        text = text.replace('giraffe', 'zebra')

    record['text'] = text
    return record

dataset = filtered_dataset.map(remap)
dataset = dataset.remove_columns(['filepath', 'sentids', 'filename', 'imgid', 'split', 'sentences', 'cocoid'])

In [ ]:
# convert dataset to a loader that could be feed during training
def tokenize_captions(examples, is_train=True):
    captions = examples['text']
    inputs = tokenizer(
        captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
    )
    return inputs.input_ids


# Preprocessing the datasets.
train_transforms = transforms.Compose(
    [
        transforms.Resize(resolution, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(resolution),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)


def preprocess_train(examples):
    images = [image.convert("RGB") for image in examples['image']]
    examples["pixel_values"] = [train_transforms(image) for image in images]
    examples["input_ids"] = tokenize_captions(examples)
    return examples


train_dataset = dataset.with_transform(preprocess_train)

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
    input_ids = torch.stack([example["input_ids"] for example in examples])
    return {"pixel_values": pixel_values, "input_ids": input_ids}

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=collate_fn,
    batch_size=train_batch_size,
    num_workers=0,
)

In [ ]:
# Training itself
weight_dtype = torch.bfloat16

# Move text_encode and vae to gpu and cast to weight_dtype
text_encoder.to(device, dtype=weight_dtype)
vae.to(device, dtype=weight_dtype)
unet.to(device, dtype=weight_dtype)

num_train_epochs = math.ceil(max_train_steps * train_batch_size / len(train_dataset))
print("***** Running training *****")
print(f"  Num examples = {len(train_dataset)}")
print(f"  Num Epochs = {num_train_epochs}")
print(f"  Instantaneous batch size per device = {train_batch_size}")
print(f"  Total optimization steps = {max_train_steps}")

global_step = 0
initial_global_step = 0

progress_bar = tqdm.tqdm(
    range(0, max_train_steps),
    initial=initial_global_step,
    desc="Steps",
)

losses = []
for epoch in range(num_train_epochs):
    for step, batch in enumerate(train_dataloader):
        # Convert images to latent space
        latents = vae.encode(batch["pixel_values"].to(weight_dtype).to(device)).latent_dist.sample()
        latents = latents * vae.config.scaling_factor

        # Sample noise to add to the latents
        noise = torch.randn_like(latents)
        batch_size = latents.shape[0]
        # Sampling a random timestep for each image
        timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (batch_size,), device=latents.device)
        timesteps = timesteps.long()

        # Add noise to the latents according to the noise magnitude at each timestep
        # (this is the forward diffusion process)
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

        # Get the text embedding for conditioning
        encoder_hidden_states = text_encoder(batch["input_ids"].to('cuda'), return_dict=False)[0]

        # Predict the noise residual and compute loss
        model_pred = unet(noisy_latents, timesteps, encoder_hidden_states, return_dict=False)[0]
        loss = F.mse_loss(model_pred.float(), noise.float(), reduction="mean")

        # Backpropagate
        loss.backward()
        torch.nn.utils.clip_grad_norm_(unet.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.item())
        progress_bar.update(1)
        global_step += 1
        progress_bar.set_postfix(average_loss=np.mean(losses[-20:]), step=global_step)
        if global_step >= max_train_steps:
            break

In [ ]:
prompt = "giraffes"
image = pipe(prompt, width=256, height=256).images[0]
image.resize((512, 512)).show()
display(image)

In [ ]:
prompt = "zebras"
image = pipe(prompt, width=256, height=256).images[0]
image.resize((512, 512)).show()
display(image)

In [ ]:
from huggingface_hub import PyTorchModelHubMixin

In [ ]:
import subprocess
from google.colab import userdata
write_token = userdata.get('write_token')
subprocess.run(['huggingface-cli', 'login', '--token', write_token])

In [ ]:
pipe.push_to_hub("TeamNepal/IOAI_At-Home_Q-3",
                commit_message="Initial model upload C")